In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
data = pd.read_csv('./data/1.2/behavior.csv', sep = ',',encoding = 'utf-8')
user = pd.read_csv('./data/1.2/user.csv', sep = ',',encoding = 'utf-8')

# R：最近购买时间

In [2]:
d1 = data[data['购买'] == 1][['用户','date']]
d1.drop_duplicates(inplace=True)
d1.sort_values(by = ['用户','date'],ascending = [True,True],inplace=True)

In [3]:
d1['是否最后一次购买'] = 1  #标记用户最后一次购买日期
for i in range(d1['用户'].count()-1):
    if d1.iloc[i, 0] == d1.iloc[i+1, 0]:
        d1.iloc[i, 2] = 0

In [4]:
d2 = d1[d1['是否最后一次购买']==1]  #仅保留用户最后一次购买日期

In [5]:
#计算R值
import datetime
for i in range(17654):
    if datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() > datetime.datetime.strptime('2014-12-12','%Y-%m-%d').date():
        d2.iloc[i, 2] = 5
    elif (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() <= datetime.datetime.strptime('2014-12-12','%Y-%m-%d').date()) \
    and (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() > datetime.datetime.strptime('2014-12-06','%Y-%m-%d').date()):
          d2.iloc[i, 2] = 4
    elif (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() <= datetime.datetime.strptime('2014-12-06','%Y-%m-%d').date()) \
    and (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() > datetime.datetime.strptime('2014-12-01','%Y-%m-%d').date()):
          d2.iloc[i, 2] = 3
    elif (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() <= datetime.datetime.strptime('2014-12-01','%Y-%m-%d').date()) \
    and (datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() > datetime.datetime.strptime('2014-11-25','%Y-%m-%d').date()):
          d2.iloc[i, 2] = 2
    elif datetime.datetime.strptime(d2.iloc[i, 1],'%Y-%m-%d').date() <= datetime.datetime.strptime('2014-11-25','%Y-%m-%d').date() :
          d2.iloc[i, 2] = 1

In [6]:
RFM = pd.merge(user,d2, how='outer',on='用户') #形成RFM数据集
RFM['是否最后一次购买'].fillna(0,inplace = True)   #未购买的消费者R=0
RFM['是否最后一次购买'] = RFM['是否最后一次购买'].astype(int)
RFM.rename(columns={'是否最后一次购买':'R值'}, inplace = True)

In [7]:
RFM = RFM.drop(['date'], axis=1) 

In [8]:
RFM

,用户,R值
0,10001082,5
1,100029775,5
2,100068031,5
3,100076491,3
4,100087810,0
...,...,...
19995,65186019,5
19996,65200062,5
19997,65222739,4
19998,65328951,5


# F：消费频率，一天算一次

In [9]:
d3 = data[data['购买'] == 1][['用户','date']]
d3.sort_values(by = ['用户','date'],ascending = [True,True],inplace=True)

In [10]:
d4 = d3.drop_duplicates()

In [11]:
se = d4.groupby(['用户'])['date'].count()

In [12]:
d5 = se.to_frame()  #se1为Series，需要转换为dataframe
d5.reset_index(level=0, inplace=True)  #索引date转为列
d5.rename(columns={'date':'购买次数'}, inplace = True)

In [13]:
d5.购买次数.max()

31

In [14]:
d5.购买次数.min()

1

In [15]:
d5['F值'] = 0
for i in range(17654):
    if d5.iloc[i, 1] > 25:
        d5.iloc[i, 2] = 5
    elif (d5.iloc[i, 1] <= 25) and (d5.iloc[i, 1] > 19):
        d5.iloc[i, 2] = 4
    elif (d5.iloc[i, 1] <= 19) and (d5.iloc[i, 1] > 13):
        d5.iloc[i, 2] = 3
    elif (d5.iloc[i, 1] <= 13) and (d5.iloc[i, 1] > 7):
        d5.iloc[i, 2] = 2
    elif (d5.iloc[i, 1] <= 7) and (d5.iloc[i, 1] > 0):
        d5.iloc[i, 2] = 1

In [16]:
RFM = pd.merge(RFM,d5[['用户','F值']], how='outer',on='用户') #形成RFM数据集

In [17]:
RFM['F值'].fillna(0,inplace = True)   #未购买的消费者R=0
RFM['F值'] = RFM['F值'].astype(int)

In [18]:
RFM

,用户,R值,F值
0,10001082,5,1
1,100029775,5,1
2,100068031,5,1
3,100076491,3,1
4,100087810,0,0
...,...,...,...
19995,65186019,5,2
19996,65200062,5,1
19997,65222739,4,2
19998,65328951,5,1


# M消费金额

In [19]:
d3

,用户,date
9473876,492,2014-12-07
9473895,492,2014-12-07
9474023,492,2014-12-07
9473947,492,2014-12-14
13919204,3726,2014-12-02
...,...,...
7672879,142442955,2014-11-23
7672983,142442955,2014-11-24
7673025,142442955,2014-11-24
7673057,142442955,2014-12-06


In [20]:
se = d3.groupby(['用户'])['date'].count()

In [21]:
d6 = se.to_frame()  #se1为Series，需要转换为dataframe
d6.reset_index(level=0, inplace=True)  #索引date转为列
d6.rename(columns={'date':'购买件数'}, inplace = True)

In [22]:
d6.购买件数.max()

342

In [23]:
d6.购买件数.min()

1

In [24]:
d6['M值'] = 0
for i in range(17654):  #17654为d6的记录数
    if d6.iloc[i, 1] > 273:
        d6.iloc[i, 2] = 5
    elif (d6.iloc[i, 1] <= 273) and (d6.iloc[i, 1] > 204):
        d6.iloc[i, 2] = 4
    elif (d6.iloc[i, 1] <= 204) and (d6.iloc[i, 1] > 135):
        d6.iloc[i, 2] = 3
    elif (d6.iloc[i, 1] <= 135) and (d6.iloc[i, 1] > 66):
        d6.iloc[i, 2] = 2
    elif (d6.iloc[i, 1] <= 66) and (d6.iloc[i, 1] > 0):
        d6.iloc[i, 2] = 1

In [25]:
RFM = pd.merge(RFM,d6[['用户','M值']], how='outer',on='用户') #形成RFM数据集
RFM['M值'].fillna(0,inplace = True)   #未购买的消费者R=0
RFM['M值'] = RFM['M值'].astype(int)

# 基于RFM分类

In [26]:
RFM['类别'] = 8
for i in range(20000):
    if (RFM.iloc[i, 1] >=3) and (RFM.iloc[i, 2] >=3) and (RFM.iloc[i, 3] >=3):
        RFM.iloc[i, 4] = 1
    elif (RFM.iloc[i, 1] >=3) and (RFM.iloc[i, 2] >=3) and (RFM.iloc[i, 3] <3):
        RFM.iloc[i, 4] = 2
    elif (RFM.iloc[i, 1] >=3) and (RFM.iloc[i, 2] <3) and (RFM.iloc[i, 3] >=3):
        RFM.iloc[i, 4] = 3
    elif (RFM.iloc[i, 1] >=3) and (RFM.iloc[i, 2] <3) and (RFM.iloc[i, 3] <3):
        RFM.iloc[i, 4] = 4
    elif (RFM.iloc[i, 1] <3) and (RFM.iloc[i, 2] >=3) and (RFM.iloc[i, 3] >=3):
        RFM.iloc[i, 4] = 5
    elif (RFM.iloc[i, 1] <3) and (RFM.iloc[i, 2] >=3) and (RFM.iloc[i, 3] <3):
        RFM.iloc[i, 4] = 6
    elif (RFM.iloc[i, 1] <3) and (RFM.iloc[i, 2] <3) and (RFM.iloc[i, 3] >=3):
        RFM.iloc[i, 4] = 7

In [27]:
RFM

,用户,R值,F值,M值,类别
0,10001082,5,1,1,4
1,100029775,5,1,1,4
2,100068031,5,1,1,4
3,100076491,3,1,1,4
4,100087810,0,0,0,8
...,...,...,...,...,...
19995,65186019,5,2,1,4
19996,65200062,5,1,1,4
19997,65222739,4,2,1,4
19998,65328951,5,1,1,4


In [28]:
RFM[RFM['类别']==1]

,用户,R值,F值,M值,类别
735,110581771,5,5,3,1
4452,33538654,5,5,5,1
4831,38714774,5,3,3,1
5108,42873856,5,4,3,1
5172,4406018,5,4,3,1
5730,51940907,5,4,3,1
5795,52671560,5,4,3,1
12127,48191511,5,5,3,1
15041,124336007,5,5,3,1
15360,128878692,5,4,3,1
